In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ekman_labeled_data.to_csv('./ekman_labeled_data.csv', index=False)

In [ ]:
df = pd.read_csv('./ekman_labeled_data.csv', converters={'sentiment_labels': lambda x: x[1:-1].split(', ')})

In [ ]:
df

In [ ]:
df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']] = pd.DataFrame(df.sentiment_labels.tolist(), index= df.index)

In [ ]:
emotion_list = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
df_dict = {}
for emotion in emotion_list:
    df_emotion = df[df[emotion]=='1']
    for emotion2 in emotion_list:
        if emotion2 != emotion:
            df_emotion = df_emotion.drop(labels=emotion2, axis=1)
    df_emotion = df_emotion.drop(labels='neutral', axis=1)
    df_emotion = df_emotion.replace(to_replace={emotion:'1'}, value=emotion)
    df_emotion = df_emotion.rename(columns={emotion: 'emotion'})
    df_dict[emotion] = df_emotion

In [ ]:
df_final

In [ ]:
df_final = df_final.drop(df_final[df_final['subreddit'] == 'Pikabu'].index)
df_final = df_final.drop(df_final[df_final['subreddit'] == 'gonewild'].index)
df_final = df_final.drop(df_final[df_final['subreddit'] == 'ChapoTrapHouse'].index)
df_final = df_final.drop(df_final[df_final['subreddit'] == 'The_Donald'].index)

In [ ]:
df_final['normalized_score'] = df_final.groupby('subreddit')['score'].transform(lambda x: (x-x.min())/(x.max()-x.min()))

In [ ]:
df_top = df_final[df_final['normalized_score'] >= df_final['normalized_score'].quantile(0.95)]

In [ ]:
len(df_final['text'].unique())

In [ ]:
sns.set_theme(style="whitegrid", color_codes=True)
histo = sns.displot(df_top, x='emotion', shrink=0.8)
histo.savefig('histogram.png')

In [ ]:
fig = sns.catplot(x='emotion', y='normalized_score', data=df_top)
fig.savefig('scatterplot.png')

In [ ]:
groups1 = df_final.groupby('emotion')
groups2 = df_top.groupby('emotion')

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2, figsize=(25, 30))
groups1['text'].count().plot.pie(ax = axes[0], subplots=True)
groups2['text'].count().plot.pie(ax = axes[1], subplots=True)

In [ ]:
df_sentiment = df_final.replace(to_replace={emotion:'joy'}, value='positive')
df_sentiment = df_sentiment.replace(to_replace={emotion:'anger'}, value='negative')
df_sentiment = df_sentiment.replace(to_replace={emotion:'sadness'}, value='negative')
df_sentiment = df_sentiment.replace(to_replace={emotion:'disgust'}, value='negative')
df_sentiment = df_sentiment.replace(to_replace={emotion:'fear'}, value='negative')
df_sentiment = df_sentiment.replace(to_replace={emotion:'surprise'}, value='ambiguous')

In [ ]:
emotion='emotion'

In [ ]:
grid = sns.FacetGrid(df_sentiment, col='subreddit', col_wrap=6)
grid.map(sns.histplot, 'emotion', shrink=0.8)

In [ ]:
count_a = df_sentiment[df_sentiment['subreddit']=='aww'].groupby(['binned', 'emotion']).size().reset_index()
count_b = df_sentiment[df_sentiment['subreddit']=='news'].groupby(['binned', 'emotion']).size().reset_index()
count_a = pd.DataFrame(count_a)
count_b = pd.DataFrame(count_b)

In [ ]:
df_aww = df_sentiment[df_sentiment['subreddit']=='aww']
df_news = df_sentiment[df_sentiment['subreddit']=='news']
df_politics = df_sentiment[df_sentiment['subreddit']=='politics']
df_worldnews = df_sentiment[df_sentiment['subreddit']=='worldnews']
df_aita = df_sentiment[df_sentiment['subreddit']=='AmItheAsshole']
df_trashy = df_sentiment[df_sentiment['subreddit']=='trashy']
df_market = df_sentiment[df_sentiment['subreddit']=='Market76']

In [ ]:
pd.concat([df_aww, df_news, df_politics, df_worldnews]).groupby(['subreddit', 'emotion'])['normalized_score'].mean()

In [ ]:
df_news.groupby('emotion')['normalized_score'].mean()